<a href="https://colab.research.google.com/github/esentoku/mywork/blob/master/temp_train_s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# google driveをマウント
from google.colab import drive
drive = drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要ライブラリをインポート
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_datasets as tfds
from keras import losses, layers, Input
#from keras.utils import np_utils
import h5py
from keras.models import Model
#from keras.layers.convolutional import MaxPooling2D
from tensorflow.keras.layers import MaxPooling2D
from keras.layers import Activation, Conv2D, Flatten, Dense,Dropout
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD, Adadelta, Adagrad, Adam, Adamax, RMSprop, Nadam
%matplotlib inline

print("done")

done


In [ ]:
# データの保存場所の指定
maindir = '/content/drive/MyDrive/Colab_Notebooks/temperature'
datadir = maindir + '/data'
tempdir = datadir + '/temp_csv'

# csvファイルへのpathを取得
l_temp = glob.glob(tempdir + '/*.csv')

# path to r4temperature
excel_path = datadir + '/r4temperature.xlsx'
r4temp = pd.read_excel(excel_path)

In [ ]:
def load_csv(fpath):
  # csvから読み込み
  temp_array = np.genfromtxt(fpath, delimiter=',', skip_header=1)

  # nanを含む列を削除
  temp_array = temp_array[:, ~np.isnan(temp_array).any(axis=0)]

  return temp_array


# load csv file to numpy array
temp_array = load_csv(l_temp[0])
temp_arrays = temp_array[np.newaxis, :, :]
for i in range(1, len(l_temp)):
  temp_array = load_csv(l_temp[i])
  temp_arrays = np.append(temp_arrays, temp_array[np.newaxis, :, :], axis=0)

# cnn用にshapeを変更
temp_arrays = temp_arrays[:, :, :, np.newaxis]


IndexError: ignored

In [ ]:
# r4tempの入力データを取得
lasers = r4temp.iloc[:, 1:9].values

# r4tempの出力データを取得
heni = r4temp.iloc[:, 14:24].values

# trainとtestデータに分割
# ここでは90%をトレーニングに利用
Xtemp_train, Xtemp_test, Xlasers_train, Xlasers_test, y_train, y_test = train_test_split(temp_arrays, lasers, heni, test_size=0.1)

# shape check
print(Xtemp_train.shape)
print(Xtemp_test.shape)
print(Xlasers_train.shape)
print(Xlasers_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# 試行錯誤した結果、下記モデルで仮設定
# 今後利用するデータ数によってはレイヤーを変更した方が良いかも

# 画像データの入力
inputs1 = Input(shape=(288, 384, 1))
x = Conv2D(4, (3, 3), padding='same', activation='relu')(inputs1)
x = Conv2D(4, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(4, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(4, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(8, (3, 3), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

# CNNデータをフラット化
x = Flatten()(x)

# r4tempのlaserデータを入力
inputs2 = Input(shape=(8))

# 画像とlaserデータをcombine
xy = layers.concatenate([x, inputs2], axis=1)

# DNN
xy = Dense(20, activation='relu')(xy)
xy = Dense(20, activation='relu')(xy)
xy = Dense(10, activation='relu')(xy)
x = Dropout(0.25)(x)

# output layer
outputs = Dense(10, activation='relu')(xy)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

In [ ]:
epochs = 300
model.compile(loss=losses.mean_squared_error,
              optimizer="adam",
              metrics=['mse'])

# モデルの訓練
history = model.fit(
    [Xtemp_train, Xlasers_train],
    y_train,
    validation_data=([Xtemp_test, Xlasers_test], y_test),
    epochs=epochs,
)

#モデルを保存する
model.save('/content/drive/MyDrive/Colab_Notebooks/temperature/model/model.h5')



In [ ]:
# plot loss and val_loss
plt.semilogy(history.history['loss'])
plt.semilogy(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

In [ ]:
# predict test data
y_pred = model.predict([Xtemp_test, Xlasers_test])
print(y_pred)

In [ ]:
# true data
y_test